In [97]:
import numpy as np
import joblib
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import math

In [98]:
def load_model(file_path):
    model=joblib.load(file_path)
    return model
        
file_path = 'D:/Paper 3 SDEE SOIL/SDEE Revision/Code/XGB_Syn.pkl'
XGB_Syn=load_model(file_path)

file_path = 'D:/Paper 3 SDEE SOIL/SDEE Revision/Code/XGB_NonSyn.pkl'
XGB_NonSyn=load_model(file_path)

file_path = 'D:/Paper 3 SDEE SOIL/SDEE Revision/Code/RF_Syn.pkl'
RF_Syn=load_model(file_path)

file_path = 'D:/Paper 3 SDEE SOIL/SDEE Revision/Code/RF_NonSyn.pkl'
RF_NonSyn=load_model(file_path)

file_path = 'D:/Paper 3 SDEE SOIL/SDEE Revision/Code/LightGBM_NonSyn.pkl'
LightGBM_NonSyn=load_model(file_path)

file_path = 'D:/Paper 3 SDEE SOIL/SDEE Revision/Code/CatB_NonSyn.pkl'
CatB_NonSyn=load_model(file_path)

file_path = 'D:/Paper 3 SDEE SOIL/SDEE Revision/Code/Kernel_RR.pkl'
Kernel_RR=load_model(file_path)

[04:23:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[04:23:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  

In [106]:
def create_TestSet(file_loc):
    df=pd.read_csv(file_loc,sep=',')
    #Getting desired features
    features=['Soil_Type','Soil_Thickness','SoilCol1','NVal','SoilCol2','SoilAvg','Actual Vs'] 
    df = df[features]
    df = df.dropna()
    dataset=df.values
    t=dataset.shape[1]-1
    X_test = dataset[:,0:t]
    y_test = dataset[:,t]
    return X_test,y_test,df

def evaluate_result(y_pred,y_test):
  print("\nR2 SCORE : %.2f"%r2_score(y_pred,y_test))
  MAE=mean_absolute_error(y_pred,y_test)
  print("\nMEAN ABSOLUTE ERROR",MAE)
  MSE=mean_squared_error(y_pred,y_test)
  print("\nROOT MEAN SQUARE ERROR",math.sqrt(MSE))

def extra_record_test(XGB_NonSyn,XGB_Syn,CatB_NonSyn,RF_NonSyn,RF_Syn,LightGBM_NonSyn,X_test,y_test,df):

  #XGB
  print("\n-----XGB_1 Syn---")
  XGB_NonSyn = XGB_NonSyn.predict(X_test)
  XGB_Syn = XGB_Syn.predict(X_test)
  print("\n------XGB Non Syn------")
  evaluate_result(y_test, XGB_NonSyn)
  print("\n----XGB Syn----")
  evaluate_result(y_test, XGB_Syn)
    
  #CatB
  CatB_NonSyn = CatB_NonSyn.predict(X_test)
  print("\n---CatB Non Syn----")
  evaluate_result(y_test, CatB_NonSyn)
    
  #RF
  RF_NonSyn = RF_NonSyn.predict(X_test)
  RF_Syn = RF_Syn.predict(X_test)
  print("\n--RF Non Syn----")
  evaluate_result(y_test, RF_NonSyn)
  print("\n----RF Syn---")
  evaluate_result(y_test, RF_Syn)

  #LGBM 
  LightGBM_NonSynthetic = LightGBM_NonSyn.predict(X_test)
  print("\n----LGBM Non Syn----")
  evaluate_result(y_test, LightGBM_NonSynthetic)

  #Averaging Layer 1
  avg_Layer_1=(XGB_Syn.astype(int)+XGB_NonSyn.astype(int)+\
            CatB_NonSyn.astype(int)+RF_NonSyn.astype(int)+\
            RF_Syn.astype(int)+\
            LightGBM_NonSynthetic.astype(int))/6
  df['Average Predict']=np.round(avg_Layer_1,decimals=0)
    
  return avg_Layer_1,df

def layer_2_stack(file_path,Kernel_RR):
  df=pd.read_csv(file_path,sep=',')
  features_L1=['Average Predict','Actual Vs']
  Syn_1=df[features_L1]
  dataset=Syn_1.values
  t=dataset.shape[1]-1
  X_test = dataset[:,0:t]
  Y_test = dataset[:,t]
  KRR_1_pred = Kernel_RR.predict(X_test)
  print("\n-----VelProfES Syn---")
  final_L2=np.round((KRR_1_pred.astype(int)+df['Average Predict'].astype(int))/2,decimals=0)
  df['VelProfES Pred']=final_L2
  evaluate_result(y_test, final_L2)
  df.to_csv(file_path,index=False)  
  return df

#Location of test file
file_path = 'D:/Paper 3 SDEE SOIL/SDEE Revision/Code/TestSet.csv'
X_test,y_test,df=create_TestSet(file_path)

final,df= extra_record_test(XGB_NonSyn,XGB_Syn,CatB_NonSyn,RF_NonSyn,RF_Syn,LightGBM_NonSyn,X_test,y_test,df)
df.to_csv(file_path,index=False)

#Test set
df=layer_2_stack(file_path,Kernel_RR)


FILE 1 :

-----XGB_1 Syn---

------XGB Non Syn------

R2 SCORE : 0.97

MEAN ABSOLUTE ERROR 14.009398040771485

ROOT MEAN SQUARE ERROR 19.931945448727465

----XGB Syn----

R2 SCORE : 0.95

MEAN ABSOLUTE ERROR 17.561705474853515

ROOT MEAN SQUARE ERROR 25.470248428541236

---CatB Non Syn----

R2 SCORE : 0.96

MEAN ABSOLUTE ERROR 15.51168132752016

ROOT MEAN SQUARE ERROR 22.241414401798945

--RF Non Syn----

R2 SCORE : 0.97

MEAN ABSOLUTE ERROR 14.07450576634068

ROOT MEAN SQUARE ERROR 19.926806443120476

----RF Syn---

R2 SCORE : 0.94

MEAN ABSOLUTE ERROR 21.104288060695094

ROOT MEAN SQUARE ERROR 26.994178407918312

----LGBM Non Syn----

R2 SCORE : 0.92

MEAN ABSOLUTE ERROR 29.162760415408375

ROOT MEAN SQUARE ERROR 31.631453788161558

-----VelProfES Syn---

R2 SCORE : 0.98

MEAN ABSOLUTE ERROR 10.67

ROOT MEAN SQUARE ERROR 15.185848675658534
